# Initialize, load modules

In [1]:
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)
%matplotlib qt5
#import awg_digital
#import awg_iq
#import data_reduce
import imp
#import fitting
#import plotting
import qsweepy.save_pkl
#import qsweepy.tomography as tomography_new
#import qsweepy.tomography_legacy as tomography
#import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(awg_iq_multi)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(qsweepy.save_pkl)
imp.reload(pulses)
imp.reload(single_shot_readout)
imp.reload(readout_classifier)
imp.reload(tomography)
imp.reload(config)
#imp.reload(tomography_new)
imp.reload(sweep)
imp.reload(qjson)

Failed loading module AWG500:  No module named 'ftd2xx'
Failed loading module Labbrick:  [WinError 126] The specified module could not be found

Failed loading module Signal_Hound_SA:  name 'signal_hound_dll' is not defined


<module 'qsweepy.qjson' from 'C:\\Users\\demouser\\Desktop\\Quantum\\qsweepy\\qjson.py'>

# some (undesirable?) settings

In [2]:
#Параметры кубитов.all keys should be str
Qubits = {'2old': {'r':{
                    'Fr': 7.29775e9,
                     'P': -30 },
              'q':{
              '00-1-01': 4.25083e+09,
              '01-1-02': 4.00871e+09,
              '00-2-02': 4.12977e+09,
                'P':None}
             },
          '2-070818':{'r':{'Fr': 7.2988e9,  'P': -30}, 'q':{'00-1-01': 4.25501e9, '00-2-02': 4.1e9, '01-1-02': 3.98e9}},
          '2':{'r':{'Fr': 7.2988e9,  'P': -30}, 'q':{'00-1-01': 4.256397e9, '00-2-02': 4.1345e9, '01-1-02': 4.01420e9}},
          '1':{'r':{'Fr': 7.2059e9, 'P': -30},'q':{ '00-1-01':4.242190e9+1.76e6, '00-2-02': 4.1e9, '01-1-02': 4.00020e9}}}
qubit_params=Qubits
qubit_id = '1'
switch_to_sample = {'1':4, '2':3}
#it should be dumped too
qubits_dump = {"qubits":qubit_params,'switch':switch_to_sample}
qjson.dump('qubits','qubit_params',qubits_dump)

# Instruments for current setup

In [3]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
#rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')

CHASSIS = 1
SLOT_IN = 11
SLOT_OUT = 7
awg = Keysight_M3202A_S('awg', CHASSIS, SLOT_OUT)
adc = Keysight_M3102A('adc', CHASSIS, SLOT_IN)

sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
lo_ex = lo1
lo_ro = pna
awg_tek=awg

0
0
0
0


In [4]:
for channel in range(0,4):
    awg.set_amplitude(.2, channel=channel)
    awg.set_offset(0.3, channel=channel)
    awg.set_output(1, channel=channel)
    awg.set_waveform(waveform=[0], channel=channel)

# Settings for current setup

In [5]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм

lo_ex_pow = 14
lo_ex.set_status(1)
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg.stop()
#awg_tek.set_clock(ex_clock) # клок всех авгшк
awg.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.
#this is zashkvar

awg_tek = awg
# channel 0 is master and triggers all others
awg.trigger_source_types = [0, 6,6,6]
awg.trigger_source_channels = [0, 4000,4000,4000] # pxi trigger 0 
awg.trigger_delays = [40, 0,0,0] # master channel should wait 400 ns for others to start
awg.trigger_behaviours = [0,4,4,4] #rising edge trigger
for channel in range(0,4):
    awg.set_amplitude(.2, channel=channel)
    awg.set_offset(0, channel=channel)
    awg.set_output(1, channel=channel)
    awg.set_waveform(waveform=[0]*awg_tek.get_nop(), channel=channel)

adc.set_input(channel=1, input=1)
adc.set_input(channel=2, input=1)
adc.set_trigger_external(channel=1)
adc.set_trigger_external(channel=2)

marker_length = 100
readout_trigger_delay = 185
trg_length = 10e-9

awg.set_marker(length=marker_length, delay=0, channel=0)
awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 1)
ro_trg.mode = 'set_delay'
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay))
awg_channels['ro_trg'] = ro_trg


100 0


In [6]:
ro_if = 75e6
lo_freq = 4.17e9
# промежуточные частоты для гетеродинной схемы new:
lo1.set_frequency(lo_freq)
pna.set_frequency(qubit_params[qubit_id]['r']['Fr']+ro_if)
ex_if = lo_freq-qubit_params[qubit_id]['q']['00-1-01']
iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 0, 1, lo_ex)
for tr,freq in qubit_params[qubit_id]['q'].items():
    iq_ex.carriers[tr] = awg_iq_multi.carrier(iq_ex)
    iq_ex.carriers[tr].set_frequency(freq)
    awg_channels['iq_ex_'+tr]=iq_ex.carriers[tr]

iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 2, 3, lo_ro)
iq_ro.carriers['iq_ro'] = awg_iq_multi.carrier(iq_ro)
iq_ro.carriers['iq_ro'].set_frequency(qubit_params[qubit_id]['r']['Fr'])
awg_channels['iq_ro'] = iq_ro.carriers['iq_ro']
readout_channels = {'iq_ro':awg_channels['iq_ro']}

pg = pulses.pulses(awg_channels)
print(ex_if*1e-6 ,lo_freq)

-73.95 4170000000.0


In [7]:
print("Excitation channel", end='\n')
iq_ex.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

Excitation channel
C:\Users\demouser\Desktop\Quantum\data/calibrations/iq-dc/ lo_freq-4.17e+09
C:\Users\demouser\Desktop\Quantum\data/calibrations/iq-rf/ frequency-4.244e+09-if-7.395e+07-sideband_id-     0
C:\Users\demouser\Desktop\Quantum\data/calibrations/iq-rf/ frequency-4.1e+09-if--7e+07-sideband_id-     0
C:\Users\demouser\Desktop\Quantum\data/calibrations/iq-rf/ frequency- 4e+09-if--1.698e+08-sideband_id-     0
Readout channel
C:\Users\demouser\Desktop\Quantum\data/calibrations/iq-dc/ lo_freq-7.281e+09
C:\Users\demouser\Desktop\Quantum\data/calibrations/iq-rf/ frequency-7.206e+09-if--7.5e+07-sideband_id-     0


In [8]:
trigger_readout_seq = [pg.p('ro_trg', 100e-9, pg.rect, 1)]

In [60]:
adc.set_nums(50)
adc.set_nop(25000)

In [61]:
from qsweepy import modem_readout
imp.reload(pulses)
imp.reload(modem_readout)
imp.reload(data_reduce)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)

modem.readout_channels = readout_channels

In [62]:
modem.demodulation(awg_channels['iq_ro'])

array([ 1.00000000+0.j        ,  0.58778525-0.80901699j,
       -0.30901699-0.95105652j, ..., -0.95105652+0.30901699j,
       -0.30901699+0.95105652j,  0.58778525+0.80901699j])

In [63]:
#plt.figure()
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0))
readout_trigger_delay = modem.calibrate_delay()[awg_channels['iq_ro']]
readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay_cycles))
readout_trigger_delay = modem.calibrate_delay()[awg_channels['iq_ro']]
readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)

2.74e-07 137.0
2e-09 1.0


In [64]:
modem.calibrate_dc()

{'iq_ro': {'coherent_background': array([-1123.54-727.82j,  -373.24-263.5j ,  -772.74-328.76j, ...,
          -391.34+263.2j ,  -701.00 -30.66j, -1057.94+338.j  ]),
  'feature': array([  3.58221114e-06 +1.22000280e-05j,
           7.79031019e-06 +1.17322303e-05j,
           5.57584774e-06 +1.59203586e-06j, ...,
          -5.57826245e-06 -1.33451524e-05j,
          -7.78977598e-06 -9.13206785e-06j,  -3.57916843e-06 +2.60976281e-06j]),
  'iq_calibration': array([ -1.02789192e-06 +9.61337613e-06j,   4.61010306e-06 +2.58665191e-06j])}}

In [65]:
modem.iq_readout_calibrations


{'iq_ro': {'coherent_background': array([-1123.54-727.82j,  -373.24-263.5j ,  -772.74-328.76j, ...,
          -391.34+263.2j ,  -701.00 -30.66j, -1057.94+338.j  ]),
  'feature': array([  3.58221114e-06 +1.22000280e-05j,
           7.79031019e-06 +1.17322303e-05j,
           5.57584774e-06 +1.59203586e-06j, ...,
          -5.57826245e-06 -1.33451524e-05j,
          -7.78977598e-06 -9.13206785e-06j,  -3.57916843e-06 +2.60976281e-06j]),
  'iq_calibration': array([ -1.02789192e-06 +9.61337613e-06j,   4.61010306e-06 +2.58665191e-06j])}}

In [66]:
modem.filters = modem.calibrated_filters

In [83]:
adc.set_nop(1000)
adc.set_nums(10000)
plt.figure('Amplitude -1')
pg.set_seq([pg.p('ro_trg', 100e-9, pg.rect, 1), pg.p('iq_ro', 10e-6, pg.rect, -1)])
I = np.real(modem.measure()['iq_ro'])
Q = np.imag(modem.measure()['iq_ro'])
plt.hist2d(I, Q)
print('')

(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)
(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)


(array([[   1.,    0.,    0.,    6.,    6.,    4.,    4.,    1.,    0.,
            0.],
        [   0.,    2.,    9.,   20.,   37.,   46.,   28.,   10.,    1.,
            0.],
        [   0.,    5.,   34.,   84.,  221.,  200.,  112.,   28.,    5.,
            0.],
        [   1.,   13.,   70.,  257.,  489.,  492.,  284.,   95.,   15.,
            2.],
        [   2.,   23.,  105.,  390.,  747.,  849.,  488.,  166.,   22.,
            7.],
        [   3.,   16.,  112.,  405.,  690.,  749.,  408.,  124.,   28.,
            4.],
        [   1.,    8.,   70.,  203.,  406.,  448.,  241.,   67.,   13.,
            1.],
        [   0.,    3.,   17.,   68.,  150.,  144.,   87.,   21.,    5.,
            1.],
        [   1.,    0.,    3.,   20.,   33.,   29.,   23.,    5.,    0.,
            0.],
        [   0.,    0.,    0.,    1.,    1.,    4.,    4.,    1.,    1.,
            0.]]),
 array([-5.70241186, -4.85178144, -4.00115102, -3.1505206 , -2.29989018,
        -1.44925976, -0.59862935,  

In [84]:
plt.figure('Amplitude +1')
pg.set_seq([pg.p('ro_trg', 100e-9, pg.rect, 1), pg.p('iq_ro', 10e-6, pg.rect, 1)])
I = np.real(modem.measure()['iq_ro'])
Q = np.imag(modem.measure()['iq_ro'])
plt.hist2d(I, Q)
print('')

(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)
(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)


(array([[  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
           1.00000000e+01,   1.30000000e+01,   6.00000000e+00,
           5.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   2.00000000e+00,   2.60000000e+01,
           6.60000000e+01,   9.20000000e+01,   6.30000000e+01,
           2.00000000e+01,   2.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  2.00000000e+00,   2.00000000e+01,   1.12000000e+02,
           2.79000000e+02,   3.85000000e+02,   2.43000000e+02,
           6.80000000e+01,   1.10000000e+01,   3.00000000e+00,
           0.00000000e+00],
        [  6.00000000e+00,   4.20000000e+01,   2.37000000e+02,
           6.87000000e+02,   8.76000000e+02,   5.53000000e+02,
           1.84000000e+02,   2.90000000e+01,   1.00000000e+00,
           0.00000000e+00],
        [  6.00000000e+00,   5.50000000e+01,   3.34000000e+02,
           8.27000000e+02,   1.03200000e+03,   6.48000000e+02,
      

In [85]:
plt.figure('Amplitude 0')
pg.set_seq([pg.p('ro_trg', 100e-9, pg.rect, 1), pg.p('iq_ro', 10e-6, pg.rect, 0)])
I = np.real(modem.measure()['iq_ro'])
Q = np.imag(modem.measure()['iq_ro'])
plt.hist2d(I, Q)
print('')

(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)
(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)


(array([[   0.,    0.,    3.,    1.,    5.,    4.,    4.,    1.,    1.,
            0.],
        [   0.,    5.,   13.,   30.,   40.,   44.,   22.,   10.,    1.,
            0.],
        [   3.,   11.,   56.,  131.,  243.,  211.,  109.,   36.,    8.,
            4.],
        [   7.,   26.,  107.,  344.,  548.,  530.,  302.,  102.,   24.,
            2.],
        [  10.,   44.,  193.,  524.,  797.,  763.,  441.,  139.,   40.,
            9.],
        [   4.,   34.,  145.,  461.,  662.,  674.,  396.,  126.,   31.,
            1.],
        [   4.,   24.,   81.,  192.,  305.,  303.,  167.,   60.,   19.,
            4.],
        [   0.,    4.,   21.,   55.,   92.,   73.,   48.,   14.,    5.,
            0.],
        [   0.,    0.,    0.,   12.,   14.,   13.,    2.,    1.,    0.,
            0.],
        [   0.,    0.,    0.,    1.,    1.,    1.,    1.,    1.,    0.,
            0.]]),
 array([-4.25042974, -3.34598376, -2.44153778, -1.53709179, -0.63264581,
         0.27180018,  1.17624616,  

In [86]:
plt.figure('Amplitude 1j')
pg.set_seq([pg.p('ro_trg', 100e-9, pg.rect, 1), pg.p('iq_ro', 10e-6, pg.rect, 1j)])
I = np.real(modem.measure()['iq_ro'])
Q = np.imag(modem.measure()['iq_ro'])
plt.hist2d(I, Q)
print('')

(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)
(10000, 1000) 1 (slice(None, None, None), slice(None, 1000, None)) (1, 25000) (1, 1000)



In [ ]:
plt.hist

In [77]:
plt.plot(np.mean(adc.measure()['Voltage'], axis=0))

C:\ProgramData\Anaconda2\envs\py36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
